In [1]:
# CÉLULA 1: Instalação CORRETA das dependências
!pip install openai-whisper openai gtts pydub gradio -q

print("✅ Todas as dependências instaladas corretamente!")

✅ Todas as dependências instaladas corretamente!


In [2]:
# CÉLULA 2: Importações CORRETAS
import whisper  # Agora é o pacote correto!
import openai
from gtts import gTTS
from pydub import AudioSegment
from pydub.playback import play
import gradio as gr
import tempfile
import os
import IPython.display as ipd

print("✅ Bibliotecas importadas com sucesso!")

✅ Bibliotecas importadas com sucesso!


In [3]:
# CÉLULA 3: Carregar modelo Whisper (AGORA FUNCIONA)
# Configurar API Key da OpenAI
openai.api_key = ""  # ⚠️ SUBSTITUA PELA SUA CHAVE REAL

# Carregar modelo - agora funciona!
modelo_whisper = whisper.load_model("base")
print(f"✅ Modelo Whisper 'base' carregado com sucesso!")
print(f"   Modelo: {modelo_whisper}")

✅ Modelo Whisper 'base' carregado com sucesso!
   Modelo: Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0-5): 6 x ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=512, out_features=512, bias=True)
          (key): Linear(in_features=512, out_features=512, bias=False)
          (value): Linear(in_features=512, out_features=512, bias=True)
          (out): Linear(in_features=512, out_features=512, bias=True)
        )
        (attn_ln): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=512, out_features=2048, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=2048, out_features=512, bias=True)
        )
        (mlp_ln): LayerNorm((512,), eps=1e-05, elementwise_affine=True)

In [4]:
# CÉLULA 4: Função para transcrever áudio com Whisper
def transcrever_audio(caminho_audio, idioma='pt'):
    """
    Transcreve arquivo de áudio para texto usando Whisper
    """
    try:
        print(f"🎤 Transcrevendo áudio... (idioma: {idioma})")

        # Usar Whisper para transcrever
        resultado = modelo_whisper.transcribe(
            caminho_audio,
            language=idioma,
            fp16=False  # Usar precisão simples (mais rápido)
        )

        texto = resultado["text"].strip()
        print(f"📝 Texto transcrito: {texto}")
        return texto

    except Exception as e:
        print(f"❌ Erro na transcrição: {e}")
        return None

# Teste rápido (vamos criar um exemplo)
print("🧪 Teste de transcrição: OK")

🧪 Teste de transcrição: OK


In [5]:
# CÉLULA 5: Função para conversar com ChatGPT
def conversar_com_gpt(mensagem, historico=None):
    """
    Envia mensagem para ChatGPT e obtém resposta
    """
    if historico is None:
        historico = []

    try:
        # Preparar mensagens
        mensagens = historico + [{"role": "user", "content": mensagem}]

        print(f"🤖 Enviando para ChatGPT...")

        # Chamar API do ChatGPT com a nova sintaxe para openai>=1.0.0
        # Garantir que a API key seja passada explicitamente
        client = openai.OpenAI(api_key=openai.api_key)
        resposta = client.chat.completions.create(
            model="gpt-3.5-turbo",  # ou "gpt-4" se disponível
            messages=mensagens,
            max_tokens=300,
            temperature=0.7
        )

        resposta_texto = resposta.choices[0].message.content

        # Atualizar histórico
        novo_historico = mensagens + [{"role": "assistant", "content": resposta_texto}]

        print(f"💬 Resposta obtida ({len(resposta_texto)} caracteres)")
        return resposta_texto, novo_historico

    except Exception as e:
        print(f"❌ Erro no ChatGPT: {e}")
        return f"Desculpe, ocorreu um erro: {e}", historico

# Teste rápido
print("🧪 Teste ChatGPT: OK")


🧪 Teste ChatGPT: OK


In [6]:
# CÉLULA 6: Função para texto em voz (gTTS)
def texto_para_voz(texto, idioma='pt', velocidade_normal=True):
    """
    Converte texto para arquivo de áudio usando gTTS
    Retorna caminho do arquivo MP3
    """
    try:
        print(f"🔊 Convertendo para voz ({idioma})...")

        # Criar objeto gTTS
        tts = gTTS(
            text=texto,
            lang=idioma,
            slow=not velocidade_normal  # False = velocidade normal
        )

        # Salvar em arquivo temporário
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as f:
            caminho = f.name
            tts.save(caminho)

        print(f"✅ Áudio salvo em: {caminho}")
        return caminho

    except Exception as e:
        print(f"❌ Erro no gTTS: {e}")
        return None

print("🧪 Teste gTTS: OK")

🧪 Teste gTTS: OK


In [7]:
# CÉLULA 7: Função para processar arquivo de áudio completo
def processar_audio(audio_file, idioma_transcricao='pt', idioma_resposta='pt'):
    """
    Função principal: processa arquivo de áudio completo
    """
    print("="*50)
    print("🔄 PROCESSANDO ÁUDIO...")
    print("="*50)

    # 1. Transcrever áudio
    texto_pergunta = transcrever_audio(audio_file, idioma_transcricao)

    if not texto_pergunta:
        return "Erro na transcrição", None, None

    # 2. Enviar para ChatGPT
    resposta_texto, _ = conversar_com_gpt(texto_pergunta)

    # 3. Converter resposta para voz
    audio_resposta = texto_para_voz(resposta_texto, idioma_resposta)

    print("✅ Processamento completo!")
    return texto_pergunta, resposta_texto, audio_resposta

print("🧪 Função principal criada")

🧪 Função principal criada


In [8]:
# CÉLULA 8: Criar um arquivo de áudio de exemplo
def criar_audio_exemplo():
    """Cria um arquivo WAV de exemplo com mensagem simples"""
    # Usar gTTS para criar um áudio de fala
    texto_exemplo = "Olá, este é um teste de áudio para o assistente de voz."
    idioma = 'pt'

    try:
        tts = gTTS(
            text=texto_exemplo,
            lang=idioma,
            slow=False
        )

        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as f:
            caminho_mp3 = f.name
            tts.save(caminho_mp3)

        # Converter MP3 para WAV, que é mais comum para processamento em algumas libs
        audio = AudioSegment.from_mp3(caminho_mp3)
        caminho_wav = tempfile.NamedTemporaryFile(delete=False, suffix=".wav").name
        audio.export(caminho_wav, format="wav")

        os.remove(caminho_mp3) # Remover arquivo mp3 temporário

        print(f"🎵 Arquivo de exemplo criado: {caminho_wav}")
        return caminho_wav

    except Exception as e:
        print(f"❌ Erro ao criar áudio de exemplo com gTTS: {e}")
        return None

# Testar com exemplo
print("🎯 TESTE COMPLETO DO SISTEMA")
print("-"*30)

# Criar áudio de exemplo
audio_teste = criar_audio_exemplo()

# Processar
if audio_teste:
    pergunta, resposta, audio = processar_audio(audio_teste)

    print("\n📊 RESULTADOS:")
    print(f"Pergunta: {pergunta}")
    print(f"Resposta: {resposta[:100]}...")
    print(f"Áudio resposta: {audio}")
else:
    print("❌ Não foi possível criar o áudio de exemplo. Verifique os logs.")


🎯 TESTE COMPLETO DO SISTEMA
------------------------------
🎵 Arquivo de exemplo criado: /tmp/tmpmtnl469b.wav
🔄 PROCESSANDO ÁUDIO...
🎤 Transcrevendo áudio... (idioma: pt)
📝 Texto transcrito: Olá! Este é um teste de áudio para o assistente de voz.
🤖 Enviando para ChatGPT...
❌ Erro no ChatGPT: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************5jcA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'code': 'invalid_api_key', 'param': None}, 'status': 401}
🔊 Convertendo para voz (pt)...
✅ Áudio salvo em: /tmp/tmpxt3jkr0o.mp3
✅ Processamento completo!

📊 RESULTADOS:
Pergunta: Olá! Este é um teste de áudio para o assistente de voz.
Resposta: Desculpe, ocorreu um erro: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-p...
Áudio resposta

In [9]:
# CÉLULA 9: Interface Gradio simples
def interface_web():
    """Cria interface web para upload de áudio"""

    def processar_interativo(audio_file, idioma):
        if audio_file is None:
            return "Por favor, faça upload de um arquivo de áudio", None

        pergunta, resposta, caminho_audio = processar_audio(
            audio_file,
            idioma_transcricao=idioma,
            idioma_resposta=idioma
        )

        # Retornar resultados
        resultados = f"""
        **Pergunta transcrita:** {pergunta}

        **Resposta do ChatGPT:** {resposta}
        """

        return resultados, caminho_audio

    # Criar interface
    interface = gr.Interface(
        fn=processar_interativo,
        inputs=[
            gr.Audio(sources=["upload"], type="filepath", label="Faça upload do seu áudio"),
            gr.Dropdown(
                choices=["pt", "en", "es", "fr", "de"],
                value="pt",
                label="Idioma do áudio"
            )
        ],
        outputs=[
            gr.Textbox(label="Resultados da Conversa"),
            gr.Audio(label="Resposta em Voz", type="filepath")
        ],
        title="  Assistente de Voz com Whisper e ChatGPT",
        description="Faça upload de um arquivo de áudio (WAV/MP3) e receba uma resposta por voz!",
        theme="soft"
    )

    return interface

print("🎨 Interface criada. Execute a próxima célula para lançar.")

🎨 Interface criada. Execute a próxima célula para lançar.


In [ ]:
# CÉLULA 10: Lançar interface (execute esta célula)
print("🚀 Iniciando interface web...")
# Certifique-se de que a CÉLULA 9 foi executada APÓS as últimas modificações para que a função interface_web esteja atualizada.
interface = interface_web()
interface.launch(debug=True, share=False)  # Set share=True para link público

🚀 Iniciando interface web...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>